In [60]:
import os
from pathlib import Path
from datetime import datetime

import numpy as np
import xarray as xr

import pyart
from dualpol import DualPolRetrieval # https://github.com/nasa/DualPol

In [19]:
#define walkspace function
def walklevel(some_dir, level=1):
    some_dir = some_dir.rstrip(os.path.sep)
    assert os.path.isdir(some_dir)
    num_sep = some_dir.count(os.path.sep)
    for root, dirs, files in os.walk(some_dir):
        yield root, dirs, files
        num_sep_this = root.count(os.path.sep)
        if num_sep + level <= num_sep_this:
            #print('hello!')
            del dirs[:]
            
def csapr_filename_time(filename):
    return datetime.strptime(filename, 'houcsapr2cfrS2.a1.%Y%m%d.%H%M%S.nc')

In [61]:
def open_sonde(path):
    ds = xr.open_dataset(path)
    # Add some variable names expected by dualpol
    snd_dict = {}
    snd_dict['z']=np.ma.MaskedArray(ds.alt.data, mask=False)
    snd_dict['T']=np.ma.MaskedArray(ds.tdry.data, mask=False)
    return snd_dict

In [67]:
basepath = '/Users/ebruning/code/TRACER-PAWS-NEXRAD-LMA/notebooks/houcsapr2cfrS2.a1/'
outpath = '/Users/ebruning/code/TRACER-PAWS-NEXRAD-LMA/notebooks/houcsapr2cfrS2.a1_proc/'

sondepath = '/data/Houston/arm-sonde/housondewnpnM1.b1.20220617.172900.cdf'

# Filter data to these times
starttime = datetime(2022,6,17,19,30)
endtime = datetime(2022,6,17,22,0)

sounding = open_sonde(sondepath)


Path(outpath).mkdir(parents=True, exist_ok=True)

In [69]:
# Download radar file
if False:
    import act
    from dotenv import load_dotenv
    load_dotenv('csapr.env')
    username = os.getenv('USER_ACT')
    token = os.getenv('TOKEN')

    datastream = "houcsapr2cfrS2.a1"
    startdate = starttime.strftime("%Y-%m-%dT%H:%M:%S")
    enddate   = endtime.strftime("%Y-%m-%dT%H:%M:%S")
    csapr_file = act.discovery.download_data(username,token, datastream, startdate, enddate)#,time='221116')

In [23]:
numfiles = 0
filepre = 'houcsapr'
csapr_files = []
for root,dirs,files in walklevel(basepath,level=1):
    for file in files:
        if file.startswith(filepre): #change start with as string
            csapr_files.append(file)
            numfiles += 1
csapr_files = sorted(csapr_files)
csapr_file_times = [csapr_filename_time(os.path.basename(fn)) for fn in csapr_files]
print(numfiles)

1818


In [21]:
filtered_csapr_scans = [fn for fn,ft in zip(csapr_files, csapr_file_times) 
                        if ((ft>=starttime) & (ft<endtime))]
print(len(filtered_csapr_scans))

446


In [63]:
def process_one_file(args):
    """ Basepath is the directory with the files given by fname;
        processed data will be saved to outpath"""
    basepath, fname, outpath = args
    radar = pyart.io.read(basepath+fname)
    retrieve = DualPolRetrieval(radar, gs=100.0, dz='reflectivity',
                                sounding=sounding,
                          dr='differential_reflectivity',
                          #dr='RD',
                          dp='differential_phase',
                          rh='copol_correlation_coeff',
                          #fhc_T_factor=2, ice_flag=True,
                          #fhc_flag=False, ice_flag=False,dsd_flag=False,precip_flag=False,liquid_ice_flag=False,
                          fhc_flag=True, ice_flag=False,dsd_flag=True,precip_flag=True,liquid_ice_flag=False,
                          kdp_window=1.8, thresh_sdp=20.0, speckle=3,
                          #kdp_window=5, thresh_sdp=20.0, speckle=3,   #..prev
                          #kdp_window=5, thresh_sdp=20.0, speckle=10,
                          kdp_method='CSU',
                          rain_method='hidro',
                          qc_flag=True, verbose=False)
    # print(outpath+fname[:-3] + '_proc.nc')
    pyart.io.write_cfradial(outpath+fname[:-3]+'_proc.nc', radar)


In [64]:
process_args = [(basepath, fn, outpath) for fn in filtered_csapr_scans]

In [65]:
from multiprocessing import Pool

with Pool(4) as pool:
    pool.map(process_one_file, process_args)

0.011951208114624023 seconds to run csu_kdp
0.0145668983459472660.01287221908569336  seconds to run csu_kdpseconds to run csu_kdp



/Users/ebruning/miniconda3/envs/lmatracer/lib/python3.7/site-packages/numpy/core/fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/ebruning/miniconda3/envs/lmatracer/lib/python3.7/site-packages/numpy/core/fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/ebruning/miniconda3/envs/lmatracer/lib/python3.7/site-packages/numpy/core/fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


0.03342580795288086 seconds to run csu_kdp
0.025266170501708984 seconds to do FHC
0.022649288177490234 seconds to do FHC
0.026762008666992188 seconds to do FHC


/Users/ebruning/miniconda3/envs/lmatracer/lib/python3.7/site-packages/numpy/core/fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/ebruning/miniconda3/envs/lmatracer/lib/python3.7/site-packages/numpy/ma/core.py:6849: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/Users/ebruning/miniconda3/envs/lmatracer/lib/python3.7/site-packages/numpy/ma/core.py:6849: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/Users/ebruning/miniconda3/envs/lmatracer/lib/python3.7/site-packages/numpy/ma/core.py:6849: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)


0.03077983856201172 seconds to do FHC


/Users/ebruning/miniconda3/envs/lmatracer/lib/python3.7/site-packages/numpy/ma/core.py:6849: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)


0.009453058242797852 seconds to run csu_kdp
0.015522956848144531 seconds to do FHC
0.012619972229003906 seconds to run csu_kdp
0.027050018310546875 seconds to run csu_kdp
0.015123844146728516 seconds to do FHC
0.023602962493896484 seconds to do FHC
0.00812983512878418 seconds to run csu_kdp
0.01378774642944336 seconds to do FHC
0.012979984283447266 seconds to run csu_kdp
0.011420965194702148 seconds to run csu_kdp
0.013972997665405273 seconds to do FHC
0.01817607879638672 seconds to do FHC
0.03379702568054199 seconds to run csu_kdp
0.013195037841796875 seconds to run csu_kdp
0.032675981521606445 seconds to do FHC
0.019685029983520508 seconds to do FHC
0.010436058044433594 seconds to run csu_kdp
0.012260198593139648 seconds to run csu_kdp
0.017282962799072266 seconds to do FHC
0.013443946838378906 seconds to do FHC
0.009109020233154297 seconds to run csu_kdp
0.015495061874389648 seconds to do FHC
0.014052152633666992 seconds to run csu_kdp
0.01890707015991211 seconds to do FHC
0.0244901